In [ ]:
#Importa le librerie necessarie per il progetto, come :
# -TensorFlow e Keras per costruire e addestrare il modello, 
# -pandas per manipolare i dati, 
# -numpy per operazioni numeriche, 
# -sklearn per dividere i dati in training e validation, 
# -PIL per manipolare le immagini e 
# -matplotlib per la visualizzazione.
import tensorflow as tf
from keras._tf_keras.keras.preprocessing.image import ImageDataGenerator
from keras.src.models import Sequential
from keras.src.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.src.callbacks import history
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from keras.src.utils import to_categorical
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# Percorsi ai dataset
train_images_path = 'data/images_training_rev1/images_training_rev1' #le immagini di training
test_images_path = 'data/images_test_rev1/images_test_rev1' #le immagini test
train_labels_path = 'solutions/training_solutions_rev1/training_solutions_rev1.csv' #le etichette csv del training

In [ ]:
#funzione che:
# Carica le immagini da una cartella, 
# le ridimensiona, 
# le normalizza (dividendo i valori dei pixel per 255.0) e raccoglie i loro ID. 
# Restituisce le immagini e i loro ID come array numpy.

def load_images_from_folder(folder, image_size, limit=None):
    images = [] #per memorizzare le immagini caricate
    image_ids = [] #per memorizzare gli id delle immagini
    for filename in os.listdir(folder)[:limit]: #itera sui file nella cartella fino al limite specificato
        img = Image.open(os.path.join(folder, filename)) #apre ogni immagine usando PIL
        if img is not None:
            img = img.resize(image_size) #la ridimensiona in base alla image_size
            img = np.array(img) / 255.0 #converte immagine in array numpy e normalizza i valori dei pixel dividendo per 255.0
            images.append(img) #aggiunge l'immagine rimensionata e normalizzata alla lista
            image_id = int(filename.split('.')[0])
            image_ids.append(image_id)  #aggiunge l'id dell'immagine nella lista
    return np.array(images), np.array(image_ids) #restituisce le lista di immagini e id convertite in array numpy

In [ ]:
# Caricamento delle immagini di addestramento
train_images, train_image_ids = load_images_from_folder(train_images_path, image_size=(128, 128)) #image_size e limit cambiabili

In [ ]:
print(train_image_ids)

In [ ]:
# Caricamento delle etichette di addestramento:
#Legge il file CSV delle etichette di addestramento e le associa agli ID delle immagini di addestramento. 
# Le etichette sono memorizzate in un array numpy.

train_labels_df = pd.read_csv(train_labels_path)
train_labels = []
for image_id in train_image_ids:
    print(image_id)
    train_labels.append(train_labels_df[train_labels_df['GalaxyID'] == image_id].iloc[:, 1:].values[0])

train_labels = np.array(train_labels)

# Creo una rete neurale convoluzionale (CNN) usando il modello 'Sequential' di Keras.
## La rete è composta da:
- Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)): un livello convoluzionale con 32 filtri, kernel di dimensione 3x3, funzione di attivazione ReLU e input shape di 256x256x3 (dimensioni dell'immagine con 3 canali colore).

- MaxPooling2D((2, 2)): un livello di max pooling con finestra di 2x2 per ridurre le dimensioni spaziali.

- Conv2D(64, (3, 3), activation='relu'): un secondo livello convoluzionale con 64 filtri.

- MaxPooling2D((2, 2)): un altro livello di max pooling.

- Flatten(): appiattisce l'output 3D a 1D per l'ingresso nei livelli densi.

- Dropout(0.5): applica il dropout con un tasso del 50% per prevenire l'overfitting.

- Dense(128, activation='relu'): un livello denso completamente connesso con 128 neuroni e funzione di attivazione ReLU.

- Dense(37, activation='sigmoid'): un livello denso con 37 neuroni e funzione di attivazione sigmoide, utilizzato per la classificazione multilabel.

In [ ]:
# Creazione del modello della rete neurale
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(37, activation='sigmoid')  # Utilizziamo sigmoid per una regressione multilabel
])


#Compila il modello usando l'ottimizzatore Adam, la loss binary_crossentropy e la metrica accuracy.
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Divisione dei dati in training(80%) e validation set(20%)
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

In [ ]:
print("imploso?")
# Addestramento del modello:
#Addestra il modello sui dati di addestramento per 10 epoche con un batch size di 64, usando il set di validazione per monitorare le prestazioni.
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

In [ ]:
# Caricamento delle immagini di test
test_images, test_image_ids = load_images_from_folder(test_images_path, image_size=(128, 128))

In [ ]:
# Predizione delle etichette per il test set
predictions = model.predict(test_images)


In [ ]:
# Creazione del DataFrame con le predizioni:
#Crea un DataFrame con le predizioni e aggiunge la colonna GalaxyID con gli ID delle immagini di test.
predictions_df = pd.DataFrame(predictions, columns=['Class' + str(i) for i in range(predictions.shape[1])])
predictions_df.insert(0, 'GalaxyID', test_image_ids)


In [ ]:
# Salvataggio delle predizioni in un file CSV
predictions_df.to_csv('galaxies_predictions_new.csv', index=False)

In [ ]:
# Mappatura delle classi alle loro descrizioni
class_descriptions = {
    0: "La galassia appare liscia, senza particolari caratteristiche.",
    1: "La galassia ha caratteristiche visibili o un disco.",
    2: "La galassia è identificata come stella o artefatto.",
    3: "La galassia è vista di taglio.",
    4: "La galassia non è vista di taglio.",
    5: "La galassia ha una barra visibile.",
    6: "La galassia non ha una barra visibile.",
    7: "La galassia ha una struttura a spirale.",
    8: "La galassia non ha una struttura a spirale.",
    9: "La galassia ha una forma rotonda.",
    10: "La galassia ha una forma squadrata.",
    11: "La galassia non ha un rigonfiamento centrale.",
    12: "La galassia ha un disco visibile di taglio.",
    13: "La galassia non ha un disco visibile di taglio.",
    14: "La galassia ha un rigonfiamento centrale arrotondato.",
    15: "La galassia ha un rigonfiamento centrale squadrato.",
    16: "La galassia non ha un rigonfiamento centrale.",
    17: "La galassia ha un braccio a spirale.",
    18: "La galassia ha due bracci a spirale.",
    19: "La galassia ha tre bracci a spirale.",
    20: "La galassia ha quattro bracci a spirale.",
    21: "La galassia ha cinque bracci a spirale.",
    22: "La galassia ha sei bracci a spirale.",
    23: "La galassia ha sette bracci a spirale.",
    24: "La galassia ha otto bracci a spirale.",
    25: "La galassia ha nove bracci a spirale.",
    26: "La galassia ha dieci bracci a spirale.",
    27: "La galassia ha un rigonfiamento centrale arrotondato.",
    28: "La galassia ha un rigonfiamento centrale squadrato.",
    29: "La galassia non ha un rigonfiamento centrale.",
    30: "La galassia ha una struttura ad anello.",
    31: "La galassia non ha una struttura ad anello.",
    32: "La galassia ha una struttura a lente o arco.",
    33: "La galassia non ha una struttura a lente o arco.",
    34: "La galassia è risultata da una fusione.",
    35: "La galassia non è risultata da una fusione.",
    36: "La galassia è sovrapposta con altre galassie o stelle."
}


In [ ]:
# Visualizzazione delle immagini di test con le classificazioni
def display_predictions(images, image_ids, predictions_df, num_images=10):
    plt.figure(figsize=(20, 10))
    for i in range(min(num_images, len(images))):
        ax = plt.subplot(2, 5, i + 1)
        plt.imshow(images[i])
        plt.title(f"GalaxyID: {image_ids[i]}")
    
        
        # Trova la classificazione con la probabilità più alta
        galaxy_id = image_ids[i]
        prediction_row = predictions_df[predictions_df['GalaxyID'] == galaxy_id]
        prediction_row = prediction_row.drop(columns=['GalaxyID']).values[0]
        top_class = np.argmax(prediction_row)
        top_class_prob = prediction_row[top_class]
        
        description = class_descriptions[top_class]
        
        plt.xlabel(f"Class: {top_class}, Prob: {top_class_prob:.2f}")
        plt.ylabel(description)
        plt.axis("off")

    plt.show()

In [ ]:
# Visualizzazione delle immagini di test con le predizioni
display_predictions(test_images, test_image_ids, predictions_df, num_images=10)

In [ ]:
# Creazione del file di testo con le descrizioni
with open('galaxies_descriptions.txt', 'w') as file:
    for i in range(len(test_images)):
        galaxy_id = test_image_ids[i]
        prediction_row = predictions_df[predictions_df['GalaxyID'] == galaxy_id]
        prediction_row = prediction_row.drop(columns=['GalaxyID']).values[0]
        top_class = np.argmax(prediction_row)
        top_class_prob = prediction_row[top_class]
        description = class_descriptions[top_class]
        file.write(f"GalaxyID: {galaxy_id}, Class: {top_class}, Prob: {top_class_prob:.2f}, Description: {description}\n")

print("File 'galaxies_descriptions.txt' creato con successo.")